In [ ]:
import sys
import os
import json
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

curr_dir = Path(os.getcwd())
root_dir = Path(curr_dir.parents[0])
sys.path.append(str(root_dir))

from agents.base_agent import BaseAgent
from agents.orchestrator_agent import OrchestratorAgent



In [ ]:
orchestrator = OrchestratorAgent(
    gemini_model="gemini-2.0-flash",
    gemini_api_key=os.getenv("GEMINI_API_KEY"),
)

In [ ]:
response = orchestrator.orchestrate(
    user_prompt="Get me to the nearest stroke center"
    
    )
    

In [ ]:
print(response)

In [ ]:
response.candidates[0].content.parts[0].text


In [ ]:

response.candidates[0].content.parts[0]


In [ ]:
print("KEY{}".format(os.getenv("GOOGLE_MAPS_API_KEY")))


In [ ]:
import requests
google_maps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")
print(google_maps_api_key)
url = f"https://www.googleapis.com/geolocation/v1/geolocate?key={google_maps_api_key}"
response = requests.post(url)

if response.status_code == 200:
    location = response.json()['location']
    lat = location['lat']
    lng = location['lng']
    accuracy = response.json()['accuracy']
    print(response)
else:
    raise Exception(f"Error: {response.text}")


In [ ]:
import whisper

model = whisper.load_model("turbo")
result = model.transcribe("audio.mp3")
print(result["text"])